<center>
<h1 class="list-group-item list-group-item-success" style="color:rgb(0,0,150)">
PREDICTION OF BODY MASS INDEX
</h1>

<img
src="https://us.123rf.com/450wm/anatolymas/anatolymas1707/anatolymas170700002/81594462-3d-small-person-reaches-for-help-button-3d-image-white-background-.jpg?ver=6"
style="width: 30%"
/>
</center>
<h3 style="font-weight: bold">Context</h3>
<p>
Body mass index is a value derived from the mass and height of a person. The BMI is defined as the body mass divided by the square of the body height, and is expressed in units of kg/m², resulting from mass in kilograms and height in metres.
</p>

<h3 style="font-weight: bold">Content</h3>

<p>
The dataset contains information about gender, height, weight and BMI index of
individuals
</p>
<p></p>

<p><span style="font-weight: bold">Gender :</span> Male / Female</p>
<p><span style="font-weight: bold">Height :</span> Age (cm)</p>
<p><span style="font-weight: bold">Height :</span> Number (cm)</p>
<p><span style="font-weight: bold">Weight :</span> Number (Kg)</p>
<p><span style="font-weight: bold">Index :</span></p>
<ul type="square">
<li>less than 18.5 - Under Weight</li>
<li>18.5 to 24.9 - Normal</li>
<li>25.0 to 29.9 - Overweight</li>
<li>30.0 to 34.9 - Obese</li>
<li>more than 35.0 - Extremely Obese</li>
</ul>

<h3 style="font-weight: bold">Contents:</h3>
<ul type="square" style="color:blue">
<li>Importing Packages</li>
<li>Importing Data</li>
<li>Analysing Data</li>
<li>Data Overview</li>
<li>Ordinal encoding</li>
<li>Model Comparsion</li>
<li>Data Transformation</li>
<li>Predicting the BMI Index</li>
<li>Evaluation Metrics</li>
</ul>

# Importing Packages

In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *
import category_encoders as ce
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_log_error,mean_absolute_error,mean_squared_error
from pandas_profiling import ProfileReport
import pickle
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Importing Data

In [2]:
df = pd.read_csv("../Data/bmi_data.csv")

# Analysing Data

In [3]:
df

,Sex,Age,Height(Inches),Weight(Pounds),BMI
0,Female,21,65.78331,112.9925,18.357646
1,Female,35,71.51521,136.4873,18.762652
2,Female,27,69.39874,153.0269,22.338985
3,Male,24,68.21660,142.3354,21.504612
4,Female,18,67.78781,144.2971,22.077669
...,...,...,...,...,...
24995,Male,35,69.50215,118.0312,17.179051
24996,Male,26,64.54826,120.1932,20.281947
24997,Female,23,64.69855,118.2655,19.864050
24998,Male,20,67.52918,132.2682,20.392540


In [4]:
df.isnull().sum()

Sex                0
Age                0
Height(Inches)    19
Weight(Pounds)    16
BMI               50
dtype: int64

In [5]:
for i in df:
    print(df[i].value_counts())

Male      12590
Female    12410
Name: Sex, dtype: int64
21    1457
27    1431
22    1429
25    1427
35    1412
31    1406
32    1393
18    1385
29    1385
23    1381
34    1378
33    1376
28    1374
19    1371
30    1364
20    1360
26    1349
24    1322
Name: Age, dtype: int64
67.98509    3
67.97769    3
68.93456    3
67.41672    3
70.04724    3
           ..
68.14995    1
69.37021    1
69.75186    1
69.18731    1
70.30910    1
Name: Height(Inches), Length: 24485, dtype: int64
124.7975    4
135.4161    3
133.5585    3
128.9968    3
121.1097    3
           ..
134.1670    1
144.0267    1
124.2302    1
131.7434    1
122.9518    1
Name: Weight(Pounds), Length: 24233, dtype: int64
20.209992    2
21.716004    2
20.579592    1
16.977697    1
18.084985    1
            ..
19.016711    1
18.022678    1
16.317830    1
15.736616    1
19.548310    1
Name: BMI, Length: 24948, dtype: int64


In [6]:
df = df.dropna()
df.reset_index(inplace=True,drop=True)

# Data Overview

In [7]:
# ProfileReport(df)

# Ordinal encoding

In [8]:
def ordinal_encoding(df,col,mapping):
    ordinal_encoder = ce.OrdinalEncoder(cols = [col],return_df = True,mapping = [{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final
mapping = {"Male":0,"Female":1}
df_final = ordinal_encoding(df,"Sex",mapping)
df_final

,Sex,Age,Height(Inches),Weight(Pounds),BMI
0,1,21,65.78331,112.9925,18.357646
1,1,35,71.51521,136.4873,18.762652
2,1,27,69.39874,153.0269,22.338985
3,0,24,68.21660,142.3354,21.504612
4,1,18,67.78781,144.2971,22.077669
...,...,...,...,...,...
24945,0,35,69.50215,118.0312,17.179051
24946,0,26,64.54826,120.1932,20.281947
24947,1,23,64.69855,118.2655,19.864050
24948,0,20,67.52918,132.2682,20.392540


In [9]:
df_final["BMI"].describe()

count    24950.000000
mean        19.321368
std          1.552091
min         13.070879
25%         18.278339
50%         19.302160
75%         20.357547
max         26.023756
Name: BMI, dtype: float64

In [10]:
df_final["Height(Inches)"] = df_final["Height(Inches)"].apply(lambda hei:hei*2.54)
df_final["Weight(Pounds)"] = df_final["Weight(Pounds)"].apply(lambda wei:wei/2.205)

In [11]:
df_final.rename(columns = {"Height(Inches)":"Height","Weight(Pounds)":"Weight"},inplace = True)

In [12]:
df_final

,Sex,Age,Height,Weight,BMI
0,1,21,167.089607,51.243764,18.357646
1,1,35,181.648633,61.899002,18.762652
2,1,27,176.272800,69.399955,22.338985
3,0,24,173.270164,64.551202,21.504612
4,1,18,172.181037,65.440862,22.077669
...,...,...,...,...,...
24945,0,35,176.535461,53.528889,17.179051
24946,0,26,163.952580,54.509388,20.281947
24947,1,23,164.334317,53.635147,19.864050
24948,0,20,171.524117,59.985578,20.392540


# Model Comparsion

In [13]:
# experiment = setup(df_final,target="BMI")

In [14]:
# compare_models()

# Data Transformation

In [15]:
X = df_final.drop("BMI",axis = 1)
Y = df_final["BMI"]

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=27,test_size = 0.2)

In [17]:
reg = ExtraTreesRegressor()
reg.fit(X_train,Y_train)


ExtraTreesRegressor()

In [18]:
reg_decision = DecisionTreeRegressor()


In [19]:
reg_decision.fit(X_train,Y_train)

DecisionTreeRegressor()

# Predicting the BMI Index

In [20]:
Y_pred = reg.predict(X_test)
Y_pred

array([18.41756013, 18.41519718, 17.82193112, ..., 15.88713748,
       18.08860229, 18.36693856])

In [21]:
Y_pred_decision = reg_decision.predict(X_test)

# Evaluation Metrics

In [22]:
r2_score(Y_pred,Y_test)
r2_score(Y_pred_decision,Y_test)


0.9980230948083236

In [23]:
print(mean_squared_error(Y_pred,Y_test))

0.0008282201558163159


In [24]:
print(mean_squared_log_error(Y_pred,Y_test))

1.6472553435790804e-06


In [25]:
pickle.dump(reg_decision,open("../models/model_bmi.pkl","wb"))

<center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWDgteV-sNXVRkc0xwyodmJt18ImebZ1UcKA&amp;usqp=CAU"></center>